In [ ]:
from pathlib import Path

In [ ]:
from toolbox_continu_inzicht import Config, DataAdapter

path = Path.cwd() / "data_sets"
config = Config(config_path=path / "demo_combine_fragility_curves_db.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

```yaml
GlobalVariables:
    rootdir: "data_sets"
    moments: [-24,0,24,48]

    fragility_curve_pipping:
        type: ci_postgresql_fragilitycurves
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        failuremechanism: "STBI"

    fragility_curve_overtopping:
        type: ci_postgresql_fragilitycurves
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        failuremechanism: "GEKB"

    fragility_curves:
        type: ci_postgresql_fragilitycurves_table
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    fragility_curves_csv:
        type: csv
        file: "test.csv"
```

In [ ]:
from toolbox_continu_inzicht.fragility_curves import CombineFragilityCurves

fragility_curves_overtopping = CombineFragilityCurves(data_adapter=data_adapter)
fragility_curves_overtopping.run(
    input=["fragility_curve_pipping", "fragility_curve_overtopping"],
    output="fragility_curves_csv",
)